In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import psycopg2

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

def connect():
    #parameter format for local use
    params = {
    'dbname':'hmda',
    'user':'roellk',
    'password':'',
    'host':'localhost',}
    #connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #set a string for connection to SQL
    try:
        conn = psycopg2.connect(**params)
        print("i'm connected")
        #return conn.cursor(cursor_factory=psycopg2.extras.DictCursor) #return a dictionary cursor object
        return conn.cursor()
    except psycopg2.Error as e: #if database connection results in an error print the following
        print("I am unable to connect to the database: ", e)


In [2]:
sql = """SELECT
    year
	,(CASE WHEN rate_spread LIKE '%NA%' OR rate_spread LIKE '%na%' OR rate_spread LIKE '% %' 
    THEN 'NaN' ELSE rate_spread END) AS rate_spread
	,lien_status
	,loan_amount
	,loan_purpose
    ,hoepa
    ,CONCAT(state_code, county_code) AS county_fips

FROM {table}
WHERE action_type = '1' AND property_type = '2' AND occupancy = '1' AND loan_purpose IN ('1', '3')
ORDER BY rate_spread DESC"""

#list of public HMDA data tables in PG
lar_tables = ["lar_2004_ffiec", "lar_2005_ffiec", "lar_2006_ffiec", "lar_2007_ffiec", "lar_2008_ffiec", "lar_2009_ffiec"
         ,"lar_2010_ffiec", "lar_2011_ffiec", "lar_2012_ffiec", "lar_2013_ffiec", "lar_2014_ffiec", "lar_2015_ffiec",
         "lar_2016_ffiec"]

In [ ]:
#Connect to database and pull data
cur = connect()
first = True
for table in lar_tables:
    sql_rs = sql.format(table=table)
    cur.execute(sql_rs)
    colnames = [desc[0] for desc in cur.description]
    results_df = pd.DataFrame(cur.fetchall(), columns=colnames)

    if first:
        rs_df = results_df.copy()
        first = False
    else:
        rs_df = pd.concat([rs_df, results_df], axis=0)
    print(rs_df.tail())
        
rs_df.head()
rs_df.to_csv("../output/base_data.csv", sep="|", index=False)

i'm connected
        year rate_spread lien_status loan_amount loan_purpose hoepa  \
219239  2004       03.00           1       00024            1     2   
219240  2004       03.00           1       00033            3     2   
219241  2004       03.00           1       00032            3     2   
219242  2004       03.00           1       00028            1     2   
219243  2004       03.00           1       00057            1     2   

       county_fips  
219239       05119  
219240       25013  
219241       12081  
219242       20107  
219243       55079  
        year rate_spread lien_status loan_amount loan_purpose hoepa  \
210341  2005       03.00           1       00150            1     2   
210342  2005       03.00           1       00023            1     2   
210343  2005       03.00           1       00024            3     2   
210344  2005       03.00           1       00024            1     2   
210345  2005       03.00           1       00018            1     2   

      